In [ ]:
 !pip3 install pyro-ppl 
 !pip install scanpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

import numpy as np
import torch
from pyro.contrib.examples.util import MNIST
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import pyro
import pyro.distributions as dist
import pyro.contrib.examples.util  # patches torchvision
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import scanpy as sc
from pandas_profiling import ProfileReport
import plotly.express as px
import plotly.io as pio
import sklearn.preprocessing


In [ ]:

assert pyro.__version__.startswith('1.8.2')
pyro.distributions.enable_validation(False)
pyro.set_rng_seed(0)
# Enable smoke test - run the notebook cells on CI.
smoke_test = 'CI' in os.environ

In [ ]:
import platform
platform.platform()
if torch.cuda.is_available:
     device = torch.device("cuda")

In [ ]:
torch.cuda.is_available()

True

In [ ]:
scdata = sc.read_h5ad("/content/drive/MyDrive/scintegration/GEX.h5ad")


In [ ]:
class GEX_Dataset(torch.utils.data.Dataset):
    
      def __init__(self, data,  scaler = None, cat_var = None):
          
            self.data = data
            
            # we need to work with the dense matrix
            self.values = data.X.todense()
            
            self.cat_var = cat_var
            
            # numerically encode the labels
            self.cat_var_data =  torch.tensor(sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]))
            
            # scale the data according to user inpt to scaler argument
            if scaler == "Standard":
                self.scaled_values = torch.tensor(sklearn.preprocessing.StandardScaler().fit_transform(self.values))
            elif scaler == "MinMax":
                self.scaled_values = torch.tensor(sklearn.preprocessing.MinMaxScaler().fit_transform(self.values))
            else:
                self.scaled_values = torch.tensor(self.values)
                
    #   return the number of genes when called 
             
      @property
      def n_features(self):
          return self.values.shape[1]
          
          
    #  A dataset class needs the following two methods to work with the dataloader class     
          
    #   return the number of cells when called
      def __len__(self):
          return len(self.data)
    
    #  return an individual cell and its label when called
      def __getitem__(self, idx):
           return self.scaled_values[idx], self.cat_var_data[idx]


In [ ]:
GEX_Dataset = GEX_Dataset(scdata, scaler = "Standard", cat_var = "batch")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
n_features = GEX_Dataset.n_features
batch_size =128

In [ ]:
train_loader = DataLoader(GEX_Dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(GEX_Dataset, batch_size=batch_size, shuffle=False)

In [ ]:
n_features = GEX_Dataset.n_features

In [ ]:
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_size, output_size):
        # call the init method of the nn.Module class
        super().__init__()
        
        # setup the two linear transformations used
        self.dfc1 = nn.Linear(z_dim, hidden_size)
        self.dfc2 = nn.Linear(hidden_size, hidden_size)
        self.dfc3 = nn.Linear(hidden_size, output_size)
        # setup the non-linearities
        self.Relu= nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    

    def forward(self, z):
        # define the forward computation on the latent z
        # first compute the hidden units
        hidden_1 = self.Relu(self.dfc1(z))
        hidden_2 = self.Relu(self.dfc2(hidden_1))
        # 
        # each is of size batch_size x n_features
        output = self.sigmoid(self.dfc3(hidden_2))
        return output

In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_size, hidden_size, input_size):
        super().__init__()
        # setup the linear transformations used
        self.efc1 = nn.Linear(input_size, hidden_size)
        self.efc2 = nn.Linear(hidden_size, hidden_size)
        self.efc3_mu = nn.Linear(hidden_size, latent_size)
        self.efc3_sigma = nn.Linear(hidden_size, latent_size)
        
        # setup the non-linearities
        self.Relu = nn.ReLU()
        self.softplus = nn.Softplus()

    def forward(self, x):
        # define the forward computation on the a cell x
        
        # then compute the hidden units
        hidden_1 = self.Relu(self.efc1(x))
        hidden_2 = self.softplus(self.efc2(hidden_1))
        
        # then return a mean vector and a (positive) square root covariance
        # each of size batch_size x z_dim
        z_loc = self.efc3_mu(hidden_2)
        z_scale = torch.exp(self.efc3_sigma(hidden_2))
        return z_loc, z_scale

In [ ]:
# define the guide (i.e. variational distribution) q(z|x)
def guide(self, x):
    # register PyTorch module `encoder` with Pyro
    pyro.module("encoder", self.encoder)
    with pyro.plate("data", x.shape[0]):
        # use the encoder to get the parameters used to define q(z|x)
        z_loc, z_scale = self.encoder(x)
        # sample the latent code z
        pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))

In [ ]:
class VAE(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, z_dim=50, hidden_dim=200, input_size = n_features, use_cuda=False):
        super().__init__()
        # create the encoder and decoder networks
        self.encoder = Encoder(z_dim, hidden_dim, input_size)
        self.decoder = Decoder(z_dim, hidden_dim, output_size=input_size)

        if use_cuda:
            # calling cuda() here will put all the parameters of
            # the encoder and decoder networks into gpu memory
            self.to(device)
        self.use_mps = use_cuda
        self.z_dim = z_dim

    # define the model p(x|z)p(z)
    def model(self, x):
        # register PyTorch module `decoder` with Pyro
        pyro.module("decoder", self.decoder)
        with pyro.plate("data", x.shape[0]):
            # setup hyperparameters for prior p(z)
            z_loc = x.new_zeros(torch.Size((x.shape[0], self.z_dim)))
            z_scale = x.new_ones(torch.Size((x.shape[0], self.z_dim)))
            # sample from prior (value will be sampled by guide when computing the ELBO)
            z = pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
            # decode the latent code z
            output = self.decoder(z)
            # score against actual images
            pyro.sample("obs", dist.Bernoulli(output).to_event(1), obs=x.reshape(-1, n_features))


    # define the guide (i.e. variational distribution) q(z|x)
    def guide(self, x):
        # register PyTorch module `encoder` with Pyro
        pyro.module("encoder", self.encoder)
        with pyro.plate("data", x.shape[0]):
            # use the encoder to get the parameters used to define q(z|x)
            z_loc, z_scale = self.encoder(x)
            # sample the latent code z
            pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))

    # define a helper function for reconstructing images
    def reconstruct_img(self, x):
        # encode image x
        z_loc, z_scale = self.encoder(x)
        # sample in latent space
        z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        output = self.decoder(z)
        return output

In [ ]:
vae = VAE(use_cuda=True)

In [ ]:
optimizer = Adam({"lr": 1.0e-3})


In [ ]:
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

In [ ]:
def train(svi, train_loader, use_cuda=True):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, _ in train_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.to(device)
        # do ELBO gradient and accumulate loss
        epoch_loss += svi.step(x)

    # return epoch loss
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    return total_epoch_loss_train

In [ ]:
def evaluate(svi, test_loader, use_cuda=True):
    # initialize loss accumulator
    test_loss = 0.
    # compute the loss over the entire test set
    for x, _ in test_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
        # compute ELBO estimate and accumulate loss
        test_loss += svi.evaluate_loss(x)
    normalizer_test = len(test_loader.dataset)
    total_epoch_loss_test = test_loss / normalizer_test
    return total_epoch_loss_test

In [ ]:
# Run options
LEARNING_RATE = 1.0e-4
USE_CUDA = True

# Run only for a single iteration for testing
NUM_EPOCHS = 1 if smoke_test else 10
TEST_FREQUENCY = 5

In [ ]:
# clear param store
pyro.clear_param_store()

# setup the VAE
vae = VAE(use_cuda=True)

# setup the optimizer
adam_args = {"lr": LEARNING_RATE}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
test_elbo = []
# training loop
for epoch in range(NUM_EPOCHS):
    total_epoch_loss_train = train(svi, train_loader, use_cuda=True)
    train_elbo.append(-total_epoch_loss_train)
    print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))

    if epoch % TEST_FREQUENCY == 0:
        # report test diagnostics
        total_epoch_loss_test = evaluate(svi, test_loader, use_cuda=True)
        test_elbo.append(-total_epoch_loss_test)
        print("[epoch %03d] average test loss: %.4f" % (epoch, total_epoch_loss_test))

[epoch 000]  average training loss: -3498.6810
[epoch 000] average test loss: -5470.4510
[epoch 001]  average training loss: -5959.9899
[epoch 002]  average training loss: -8625.9631
[epoch 003]  average training loss: -9630.4942
[epoch 004]  average training loss: -10156.4808
[epoch 005]  average training loss: -10504.4505
[epoch 005] average test loss: -10694.1128
[epoch 006]  average training loss: -10786.4320
[epoch 007]  average training loss: -11022.7556
[epoch 008]  average training loss: -11221.0258
[epoch 009]  average training loss: -11377.7079


In [ ]:
vae.reconstruct_img()

TypeError: ignored